# Pipeline 
- Insertar Imagen
- Procesar Imagen
- Separar en líneas
- Enviar a los distintos OCRs
- Correcciones 
- Decisión Final

## Insertar Imágenes

In [1]:
import os


input_folder = "data/input"
os.makedirs(input_folder, exist_ok=True)
output_folder = "data/output_preprocessing"
os.makedirs(output_folder, exist_ok=True)

# Procesar Imágenes

In [2]:
# from Preprocessing.pipeline2 import cleanup_image

# cleanup_image(input_folder, output_folder)

## Separar en líneas

## Enviar a los distintos OCRs

In [3]:
import os
from typing import List
from Recognition.Kraken.kraken_model import get_transcription

processed_photos = os.listdir(output_folder)

transcriptions = []

for photo in processed_photos:
    transcriptions.append(get_transcription(output_folder + '/' + photo))
    
# output_ocr = 'data/output_ocr'
# transcriptions = os.listdir(output_ocr)
# results_from_ocrs:List[List[List[str]]] = []

ocr_results = []

# Para cada imagen tenemos una transcripcion
for image in transcriptions:
    lines = []
    for model in image:         # La transcripcion viene del modo model: x text: y
        text = model['text']
        lines = text.split('\n')        # Separamos el texto por lineas
    ocr_results.append(lines)


## Correcciones

In [4]:
from serialization.serialization import correction, combine, load_corrector_model
import google.generativeai as genai
from dotenv import load_dotenv

# results_from_ocr = [
#     [
#         "Essta es una prueva de ectraccion de teexto. Connoscida cosa sea a todos los queesta carta uieren como yo don fffffff por la gracia de dios hey de Castiella.",
#         "Asta carta fue escrita en el anno del senyor de 1493, y  cosas de gran ymportancia para los reynos."
#     ],
#     [
#         "Essa s una prsueva de etraccion de . Connoda cosa sea a tdos as los qeesta carta ueren como yo don Fferrando por la gracia de d hey de Castiella.",
#         "Asta s fue escrita en el ano de seyor de 143, y consigna cossdas de gran ymportana para los reynos."
#     ],
# ]

# Cargar claves de entorno para Gemini
load_dotenv()
GENAI_API_KEY = os.getenv("GENAI_API_KEY")
genai.configure(api_key=GENAI_API_KEY)

spacy_model = "es_core_news_sm"
frequency_dictionary_path = "serialization/spanish_frequency_dictionary.txt"

# Cargar modelo SymSpell y Spacy
nlp, sym_spell = load_corrector_model(
    spacy_model=spacy_model,
    frequency_dictionary_path=frequency_dictionary_path,
    max_edit_distance=3,
    prefix_length=7
)

gemini_model_name="gemini-2.0-flash-exp"

# Crear el sistema de instrucciones para Gemini
chat = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction=[
        """
        Eres un asistente que ayuda a refinar textos extraídos de manuscritos antiguos en español.Los textos son documentos históricos
        extraídos mediante varios modelos OCR, por lo que pueden contener errores típicos del procesamiento, palabras incorrectas y poca coherencia. Los documentos son actas capitulares de La Habana, Cuba; de los siglos XVI y XVII, escritas en letra procesal cortesana, con términos de la época y ambiguedades, ten esto en cuenta y corrigelo, cambia las palabras incorrectas por palabras parecidas o cercanas sintácticamente que tengan sentido en este contexto y dame un texto lo mas coherente pero fiel al original posible.
        """
    ]
).start_chat(history=[])


KeyboardInterrupt: 

In [ ]:
results_from_corrections: List[List[str]] = []

for document in ocr_results:
    aux: List[str] = []
    for line in document:
        aux.append(correction(line, nlp, sym_spell, chat))
    results_from_corrections.append(aux)
    
print(results_from_corrections)

## Decisión Final

In [ ]:
result:List[str] = []

# Crear el modelo generativo y configurar el sistema de instrucciones
chat2 = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction="""
    Eres un asistente que ayuda a refinar textos. Analiza las siguientes representaciones de una línea de texto,
    y reescribe solo una en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
    extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
    cartas en Cuba de los siglos XV y XVI, manten el español antiguo en tu respuesta.
    """
).start_chat(history=[])

for it in range(len(ocr_results[0])):
    aux: List[str] = []
    for i in range(len(ocr_results)):
        aux.append(results_from_corrections[i][it])
    
    result.append(combine(aux, chat2))

print("RESULT FINAL")
print(result)